# 📊 Clutch Revenue: Predicción del Valor Comercial en la NBA

Este notebook está organizado en secciones para:

1. **Cargar y limpiar** los datos de partidos, jugadores y jugadas.
2. **Filtrar jugadas “clutch”** (últimos 5 minutos, periodo ≥ 4, marcador ±5).
3. **Explorar esas jugadas**: tipos de eventos, jugadores más activos, etc.
4. **Calcular métricas de tiro clutch**: tiros intentados, anotados, eficiencia.
5. **Visualizar** los resultados para storytelling.


In [12]:
# 3.1. Importar librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Para mostrar gráficos inline si usas Jupyter:
%matplotlib inline

# 3.2. Ajustes de estilo (opcional)
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)


In [13]:
# 4.1. Columnas y dtypes para play_by_play.csv
cols_plays = [
    "game_id",
    "eventmsgtype",
    "period",
    "pctimestring",
    "scoremargin",
    "player1_id",
    "player1_name",
    "homedescription",
    "visitordescription"
]

dtypes_plays = {
    "game_id": "Int32",
    "eventmsgtype": "UInt8",
    "period": "UInt8",
    "player1_id": "Int32",
    "player1_name": "object",
    "homedescription": "object",
    "visitordescription": "object"
}

# 4.2. Columnas y dtypes para game.csv
cols_games = [
    "game_id",
    "season_id",
    "game_date",
    "team_id_home",
    "team_id_away",
    "pts_home",
    "pts_away"
]

dtypes_games = {
    "game_id": "Int32",
    "season_id": "object",
    "team_id_home": "Int16",
    "team_id_away": "Int16",
    "pts_home": "Int16",
    "pts_away": "Int16"
}

# 4.3. Columnas y dtypes para player.csv
cols_players = ["id", "full_name", "first_name", "last_name", "is_active"]
dtypes_players = {
    "id": "Int32",
    "full_name": "object",
    "first_name": "object",
    "last_name": "object",
    "is_active": "boolean"
}

# 4.4. (Opcional) Columnas y dtypes para other_stats.csv si fuera necesario
cols_other = [
    "game_id",
    "team_id_home",
    "team_id_away",
    "pts_paint_home",
    "pts_paint_away",
    "team_turnovers_home",
    "team_turnovers_away",
    "team_rebounds_home",
    "team_rebounds_away"
]

dtypes_other = {
    "game_id": "Int32",
    "team_id_home": "Int16",
    "team_id_away": "Int16",
    "pts_paint_home": "Int16",
    "pts_paint_away": "Int16",
    "team_turnovers_home": "Int16",
    "team_turnovers_away": "Int16",
    "team_rebounds_home": "Int16",
    "team_rebounds_away": "Int16"
}


In [14]:
def cargar_partidos(path_game="game.csv"):
    """
    Lee game.csv y devuelve:
      - df_games: DataFrame completo con columnas definidas en cols_games.
      - dict_game_to_season: mapeo game_id → season_id.
    """
    try:
        df_games = pd.read_csv(
            path_game,
            usecols=cols_games,
            dtype=dtypes_games,
            parse_dates=["game_date"]
        )
        print(f"Cargados {len(df_games)} registros de partidos.")
    except Exception as e:
        print(f"Error al cargar {path_game}: {e}")
        df_games = pd.DataFrame(columns=cols_games)

    # Crear diccionario game_id → season_id
    if not df_games.empty:
        dict_game_to_season = df_games.set_index("game_id")["season_id"].to_dict()
    else:
        dict_game_to_season = {}

    return df_games, dict_game_to_season

# Uso:
df_games, game_to_season = cargar_partidos("game.csv")


Cargados 65698 registros de partidos.


In [15]:
def cargar_jugadores(path_player="player.csv"):
    """
    Lee player.csv y devuelve un DataFrame con info de jugadores.
    """
    try:
        df_players = pd.read_csv(
            path_player,
            usecols=cols_players,
            dtype=dtypes_players
        )
        print(f"Cargados {len(df_players)} jugadores.")
    except Exception as e:
        print(f"Error al cargar {path_player}: {e}")
        df_players = pd.DataFrame(columns=cols_players)

    return df_players

# Uso:
df_players = cargar_jugadores("player.csv")


Cargados 4831 jugadores.


In [16]:
def procesar_jugadas_clutch(path_plays="play_by_play.csv", dict_game_to_season=None):
    """
    Recorre play_by_play.csv en chunks, calcula 'seconds_remaining',
    asigna 'season_id' y retorna un DataFrame con SOLO jugadas clutch:
      - period >= 4
      - seconds_remaining <= 300
      - abs(scoremargin) <= 5
    """
    df_clutch = pd.DataFrame(columns=cols_plays + ["seconds_remaining", "season_id"])

    try:
        chunk_iter = pd.read_csv(
            path_plays,
            usecols=cols_plays,
            dtype=dtypes_plays,
            chunksize=400_000
        )

        for i, chunk in enumerate(chunk_iter, start=1):
            # ----------------------------------------------------
            # 5.3.1. Convertir 'scoremargin' a numérico (puede contener NaN)
            chunk["scoremargin"] = pd.to_numeric(chunk["scoremargin"], errors="coerce").astype("Int16")

            # 5.3.2. Calcular 'seconds_remaining' solo cuando haya formato "MM:SS"
            #        Para cualquier valor no numérico (ej. "TIE") devolvemos NaN.
            chunk["seconds_remaining"] = pd.NA
            mask_timevalido = chunk["pctimestring"].str.match(r"^\d{1,2}:\d{2}$", na=False)

            mins = pd.to_numeric(chunk.loc[mask_timevalido, "pctimestring"].str.split(":", n=1).str[0], errors="coerce").astype("Int32")
            secs = pd.to_numeric(chunk.loc[mask_timevalido, "pctimestring"].str.split(":", n=1).str[1], errors="coerce").astype("Int32")
            chunk.loc[mask_timevalido, "seconds_remaining"] = mins * 60 + secs

            # 5.3.3. Asignar 'season_id' usando el diccionario
            if dict_game_to_season is not None:
                chunk["season_id"] = chunk["game_id"].map(dict_game_to_season)
            else:
                chunk["season_id"] = pd.NA

            # 5.3.4. Filtrar jugadas "clutch"
            mask_clutch = (
                (chunk["period"] >= 4) &
                (chunk["seconds_remaining"] <= 300) &
                (chunk["scoremargin"].abs() <= 5)
            )
            df_chunk_clutch = chunk.loc[mask_clutch].copy()

            # 5.3.5. Acumular resultados
            df_clutch = pd.concat([df_clutch, df_chunk_clutch], ignore_index=True)
            print(f"Chunk {i:>2} procesado → Total jugadas clutch acum.: {len(df_clutch)}")

            print(chunk[["period", "pctimestring", "seconds_remaining", "scoremargin"]].head(10))
            print("period >= 4:", (chunk["period"] >= 4).sum())
            print("seconds_remaining <= 300:", (chunk["seconds_remaining"] <= 300).sum())
            print("scoremargin.abs() <= 5:", (chunk["scoremargin"].abs() <= 5).sum())
            print("Todos los filtros juntos:", mask_clutch.sum())
 # para que solo procese el primer chunk y puedas ver el resultado

        print(f"\n✔️  Total jugadas 'clutch' encontradas: {len(df_clutch)}\n")
    except Exception as e:
        print(f"Error al procesar {path_plays}: {e}")

    return df_clutch

# Uso:
df_clutch = procesar_jugadas_clutch("play_by_play.csv", game_to_season)


/var/folders/sc/9fn1kvh90nq2n1t913qbts5w0000gq/T/ipykernel_6870/3532253307.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_clutch = pd.concat([df_clutch, df_chunk_clutch], ignore_index=True)


Chunk  1 procesado → Total jugadas clutch acum.: 4834
   period pctimestring seconds_remaining  scoremargin
0       1        12:00               720         <NA>
1       1        12:00               720         <NA>
2       1        11:45               705         <NA>
3       1        11:43               703         <NA>
4       1        11:29               689         <NA>
5       1        11:27               687         <NA>
6       1        11:14               674         <NA>
7       1        11:08               668         <NA>
8       1        10:49               649         <NA>
9       1        10:49               649         <NA>
period >= 4: 110097
seconds_remaining <= 300: 187743
scoremargin.abs() <= 5: 44043
Todos los filtros juntos: 4834
Chunk  2 procesado → Total jugadas clutch acum.: 9227
        period pctimestring seconds_remaining  scoremargin
400000       2        11:44               704         <NA>
400001       2        11:30               690         <NA>
400002 

---
# 🕵️‍♂️ 6. Análisis Exploratorio de Datos “Clutch”

**Contenidos**
6.1. Tipos de eventos en momentos clutch
6.2. Top jugadores con más acciones clutch
6.3. Distribución por periodo
6.4. Relación evento vs. margen de puntos
---
